##### Executive Summary

In this notebook, we train Machine Learning (ML) models on the SEED dataset using augmented features (obtained via MATLAB codes).

Each model in ML and DL pipeline has been evaluated via five fold (stratified) cross validation. Then the best model is selected and its performance on the test set is evaluated. Note that the test set is fixed accross pipelines and corresponds to the 20% of the data. F1 scores of Machine Learning models on the test set is shared on the table below:

| Model | Test F1 Score (%) |
| ----- | --------------- |
| MLP | 66.3 |
| RF | 67.6 |
| KNN | 67.6 |
| Log - Reg | 77.6 |
| SVM | 77.7 |
| XGBoost | 79.8 |

According to our experiments, XGBoost algorithm outperformed the remainining ML pipelines. Here note that each ML model has been saved under the directory `models/` by the dataset's name and model's name (e.g. knn). Also, you can find cross validation results under the `searchs/` directory.

In orted to advance the performance of our models, we have trained a CNN_LSTM on the 10-timestep averaged raw data and extracted the 15992 dimensional embeddings. Note that these embeddings are optimized to model the discrepancy of each class. Then we have concatenated the 16150 dimensional MATLAB features with 15992 dimensional optimized embeddings. The resulting 32142 dimensional feature vectors are given as input to ML models. We name the models trained using this "augmented" approach CNN_LSTM based models. Below we share the performance of the augmented models:

| Model (Combined features) | Test F1 Score (%) |
| ----- | --------------- |
| MLP | 82.5 |
| RF | 80.3 |
| KNN | 71 |
| Log - Reg | 88.7 |
| SVM | 85.8 |
| XGBoost | 89.2 |

Augmented models and their corresponding searchs can be found on `models/` and `searchs/` directory with the prefix `seed-combined`.

In [ ]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from itertools import product
from imblearn.over_sampling import ADASYN
from imblearn.pipeline import make_pipeline, Pipeline

In [ ]:
import torch
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import xgboost

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
from tensorflow.keras.utils import to_categorical
from scipy.io import loadmat

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def cnn_lstm_model(T, F, num_feats=16, num_layers=2, p=0.5, num_labels=1, act='sigmoid'):
    inp = keras.Input(shape=(T, F))
    for l in range(num_layers): 
        x = layers.BatchNormalization(axis=-1)(x if l else inp)
        x = layers.Conv1D(num_feats,2, activation='relu')(x)
        x = layers.Conv1D(num_feats,2, activation='relu')(x)
        x = layers.Dropout(p)(x)
        x = layers.MaxPooling1D(2, data_format="channels_last")(x)
        x = layers.LSTM(100, return_sequences=True)(x)  # Return sequences for the next LSTM
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(64)(x)
        x = layers.Dropout(0.25)(x)
        x = layers.Dense(8,activation='relu')(x)
    x = layers.Flatten()(x)
    out = layers.Dense(num_labels, activation=act)(x)
    return keras.Model(inputs=inp, outputs=out)

def teleport(data):
    mel = Spectrogram(n_fft=16, normalized=True)
    R = mel(torch.Tensor(data.transpose([0, 2, 1]))).numpy()
    N, F1, F2, T = R.shape
    return R.reshape(N, F1*F2, T)

def cv_keras(model, X, y, n=5, verbose=0):
    skf = StratifiedKFold(random_state=0, n_splits=n, shuffle=True)
    logs = {}
    score = 0
    for i, (train_index, test_index) in enumerate(skf.split(X, y.argmax(axis=-1))):
        history = model.fit(X[train_index], 
                            y[train_index], 
                            validation_data=(X[test_index], y[test_index]), 
                            epochs=50, 
                            shuffle=True,
                            callbacks=[
                                tf.keras.callbacks.EarlyStopping(patience=5),
                                tf.keras.callbacks.ReduceLROnPlateau(patience=3)
                            ],
                            verbose=verbose)

        score += np.mean(history.history["val_f1_score"][-2:]) / n
    return score

In [ ]:
class F1_Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.f1 = self.add_weight(name='f1', initializer='zeros')
        self.precision_fn = keras.metrics.Precision(thresholds=0.5)
        self.recall_fn = keras.metrics.Recall(thresholds=0.5)

    def update_state(self, y_atrue, y_pred, sample_weight=None):
        p = self.precision_fn(y_true, y_pred)
        r = self.recall_fn(y_true, y_pred)
        # since f1 is a variable, we use assign
        self.f1.assign(2 * ((p * r) / (p + r + 1e-6)))

    def result(self):
        return self.f1

    def reset_states(self):
        # we also need to reset the state of the precision and recall objects
        self.precision_fn.reset_states()
        self.recall_fn.reset_states()
        self.f1.assign(0);

In [ ]:
path= "C:\\Users\\loitp\\Downloads\\ftr_arr.mat"
data = loadmat(path)['ftr_arr']

In [ ]:
labels=np.array

## 1. Label Balance Check

In [ ]:
print(f"Label balance:\nClass -1: {round((labels == -1).sum() / len(labels), 2)}\nClass  0: {round((labels == 0).sum() / len(labels), 2)}\nClass  1: {round((labels == 1).sum() / len(labels), 2)}")

In [ ]:
DATA_TYPE = "seed-feats"

# 2. Training Machine Learning models

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(data, labels,
                                                test_size=0.2,
                                                stratify=labels,
                                                random_state =0)

In [ ]:
scaler = StandardScaler() # Scaling the data. Use another scaling function if you want.
Xtr = scaler.fit_transform(Xtrain)
Xte = scaler.fit_transform(Xtest)

In [ ]:
N, F = Xtr.shape

In [ ]:
steps = [
    ('logreg', [('clf', LogisticRegression(random_state=0, max_iter=1000))]),
    ('logreg-pca', [('pca', PCA(random_state=0)), ('clf', LogisticRegression(random_state=0, max_iter=1000))]),
    ('knn', [('clf', KNeighborsClassifier())]),
    ('knn-pca', [('pca', PCA(random_state=0)), ('clf', KNeighborsClassifier())]),
    ('svm', [('clf', SVC(random_state=0))]),
    ('svm-pca', [('pca', PCA(random_state=0)), ('clf', SVC(random_state=0))]),
    ('neuralnet', [('clf', MLPClassifier(random_state=0, learning_rate="adaptive", max_iter=1000, early_stopping=True))]),
    ('neuralnet-pca', [('pca', PCA(random_state=0)), ('clf', MLPClassifier(random_state=0, learning_rate="adaptive", max_iter=1000, early_stopping=True))]),
    ('rf', [('clf', RandomForestClassifier(random_state=0))]),
    ('rf-pca', [('pca', PCA(random_state=0)), ('clf', RandomForestClassifier(random_state=0))])
]

In [ ]:
grids = [
    {"clf__C": np.logspace(-6, 0, 10)},
    {"pca__n_components": [5, 10, 25, 50, 100], "clf__C": np.logspace(-6, 0, 10)},
    {"clf__n_neighbors": [2, 5, 10, 25, 50]},
    {"pca__n_components": [5, 10, 25, 50, 100], "clf__n_neighbors": [2, 5, 10, 25, 50]},
    {"clf__C": np.logspace(-6, 0, 10), "clf__kernel": ["linear", "poly", "rbf", "sigmoid"]},
    {"pca__n_components": [5, 10, 25, 50, 100], "clf__C": np.logspace(-6, 0, 10), "clf__kernel": ["linear", "poly", "rbf", "sigmoid"]},
    {"clf__hidden_layer_sizes": [*product([8, 16, 32, 64], [8, 16, 32, 64])], 'clf__learning_rate_init': np.logspace(-4, 0, 4), 'clf__alpha': np.logspace(-4, 0, 4)},
    {"pca__n_components": [5, 10, 25, 50, 100], "clf__hidden_layer_sizes": [*product([8, 16, 32, 64], [8, 16, 32, 64])], 'clf__learning_rate_init': np.logspace(-4, 0, 4), 'clf__alpha': np.logspace(-4, 0, 4)},
    {"clf__n_estimators": [10, 25, 50, 100, 200]},
    {"pca__n_components": [5, 10, 25, 50, 100], "clf__n_estimators": [10, 25, 50, 100, 200], "clf__max_depth": [2, 3, 5, None], "clf_min_samples_leaf": [1, 3, 5, 10, 20]}
]

In [ ]:
with open(f"...\\models{DATA_TYPE}-scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [ ]:
results = []
for (name, pipe), grid in zip(steps, grids):
    if name.endswith('-pca'):
        continue
    print(f"@{name}")
    pipe = [('selector', SelectKBest(mutual_info_classif))] + pipe
    grid['selector__k'] = np.logspace(0.7, np.log10(F), 5).astype(int)
    pipeline = Pipeline(pipe)
    grid = GridSearchCV(pipeline, grid, cv=5, scoring="f1_macro", n_jobs=-1)
    grid.fit(Xtr, ytrain)
    pd.DataFrame(grid.cv_results_).sort_values("rank_test_score").to_csv(f"...\\searchs{DATA_TYPE}-{name}.csv")
    clf = grid.best_estimator_
    clf.fit(Xtr, ytrain)
    with open(f"...\\models{DATA_TYPE}-{name}.pkl", "wb") as f:
        pickle.dump(clf, f)
    y_tr_hat = clf.predict(Xtr)
    y_te_hat = clf.predict(Xte)
    res = {"name": name}
    for phase, pred, true in [("train", y_tr_hat, ytrain), ("test", y_te_hat, ytest)]:
        for f in [accuracy_score, precision_score, recall_score, f1_score]:
            try: 
                res[f"{phase}-{f.__name__}"] = f(true, pred, average="macro")
            except:
                res[f"{phase}-{f.__name__}"] = f(true, pred)
        
    results.append(res)
    print(json.dumps(res, indent=2))
    print("-"*50)
    print()

### XGBOOST

In [ ]:
from xgboost import XGBClassifier
name = "xgb"
pipe = [('clf', XGBClassifier(n_estimators=100, objective='multi:softmax', n_jobs=-1,
                    silent=True, nthread=4))]
grid = {
        'clf__n_estimators': [100, 200, 300, 400, 500],
        'clf__max_depth': [3, 4, 5],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9],
        'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
        'min_child_weight': [1, 2, 3, 4, 5],
        'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05],
        'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
        }

In [ ]:
pipe = [('selector', SelectKBest(mutual_info_classif))] + pipe
grid['selector__k'] = np.logspace(0.7, np.log10(F), 5).astype(int)
pipeline = Pipeline(pipe)
try:
    grid = GridSearchCV(pipeline, grid, scoring="f1_macro", n_jobs=-1, verbose=4)
    grid.fit(Xtr, ytrain)
    pd.DataFrame(grid.cv_results_).sort_values("rank_test_score").to_csv(f"searchs/{DATA_TYPE}-{name}.csv")
    clf = grid.best_estimator_
    clf.fit(Xtr, ytrain)
    with open(f"models/{DATA_TYPE}-{name}.pkl", "wb") as f:
        pickle.dump(clf, f)
    y_tr_hat = clf.predict(Xtr)
    y_te_hat = clf.predict(Xte)
    res = {"name": name}
    for phase, pred, true in [("train", y_tr_hat, ytrain), ("test", y_te_hat, ytest)]:
        for f in [accuracy_score, precision_score, recall_score, f1_score]:
            try: 
                res[f"{phase}-{f.__name__}"] = f(true, pred, average="macro")
            except:
                res[f"{phase}-{f.__name__}"] = f(true, pred)

    print(json.dumps(res, indent=2))
    print("-"*50)
    print()
except Exception as e:
    print(str(e))

# 3. Deep Learning Models for Automated Features

In [ ]:
data_avgd = torch.nn.AvgPool1d(10)(torch.Tensor(np.load("data/other/assembled_data.npy"))).numpy()

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(data_avgd, labels,
                                                test_size=0.2,
                                                stratify=labels,
                                                random_state =0)
Xtrain = np.transpose(Xtrain, [0, 2, 1])
Xtest = np.transpose(Xtest, [0, 2, 1])
Xtrain.shape, Xtest.shape

In [ ]:
ytrain = to_categorical(ytrain, num_classes=3)
ytest = to_categorical(ytest, num_classes=3)

In [ ]:
_, T, F = Xtrain.shape

In [ ]:
DATA_TYPE = "seed_cnn_lstm"
tf.keras.backend.clear_session()

In [ ]:
model = cnn_lstm_model(*Xtrain.shape[1:], num_feats=8, num_layers=1, act='softmax', num_labels=3)
model.compile(optimizer=keras.optimizers.Adam(),
                  loss="categorical_crossentropy", 
                  metrics=["accuracy", keras.metrics.Precision(),
                           keras.metrics.Recall(), 
                           F1_Score()])
history = model.fit(Xtrain, 
                            ytrain, 
                            validation_data=(Xtest, ytest), 
                            epochs=50, 
                            shuffle=True,
                            callbacks=[
                                tf.keras.callbacks.EarlyStopping(patience=5),
                                tf.keras.callbacks.ReduceLROnPlateau(patience=3)
                            ],
                            verbose=1)
model.save(f".../models/{DATA_TYPE}-cnn.h5")

In [ ]:
model.summary()

In [ ]:
layer_name = 'flatten'
intermediate_layer_model = keras.Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)

In [ ]:
cnn_feats = intermediate_layer_model.predict(data_avgd.transpose([0, 2, 1]))

In [ ]:
combined_feats = np.concatenate([data[:, :], cnn_feats], axis=-1)

In [ ]:
del model
del Xtrain
del Xtest
del data_avgd
tf.keras.backend.clear_session()

# 4. Handcrafted Features + CNN_LSTM Features

In [ ]:
DATA_TYPE = "seed-combined"

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(combined_feats
                                                ,
                                                labels,
                                                test_size=0.2,
                                                stratify=labels,
                                                random_state =0)

In [ ]:
scaler = StandardScaler() # scaling the data
Xtr = scaler.fit_transform(Xtrain)
Xte = scaler.transform(Xtest)

In [ ]:
steps = [
    ('logreg', [('clf', LogisticRegression(random_state=0, max_iter=1000))]),
    ('logreg-pca', [('pca', PCA(random_state=0)), ('clf', LogisticRegression(random_state=0, max_iter=1000))]),
    ('knn', [('clf', KNeighborsClassifier())]),
    ('knn-pca', [('pca', PCA(random_state=0)), ('clf', KNeighborsClassifier())]),
    ('svm', [('clf', SVC(random_state=0))]),
    ('svm-pca', [('pca', PCA(random_state=0)), ('clf', SVC(random_state=0))]),
    ('neuralnet', [('clf', MLPClassifier(random_state=0, learning_rate="adaptive", max_iter=1000, early_stopping=True))]),
    ('neuralnet-pca', [('pca', PCA(random_state=0)), ('clf', MLPClassifier(random_state=0, learning_rate="adaptive", max_iter=1000, early_stopping=True))]),
    ('rf', [('clf', RandomForestClassifier(random_state=0))]),
    ('rf-pca', [('pca', PCA(random_state=0)), ('clf', RandomForestClassifier(random_state=0))])
]

grids = [
    {"clf__C": np.logspace(-6, 0, 10)},
    {"pca__n_components": [5, 10, 25, 50, 100], "clf__C": np.logspace(-6, 0, 10)},
    {"clf__n_neighbors": [2]},
    {"pca__n_components": [5, 10, 25, 50, 100], "clf__n_neighbors": [2]},
    {"clf__C": np.logspace(-6, 0, 10), "clf__kernel": ["linear"]},
    {"pca__n_components": [5, 10, 25, 50, 100], "clf__C": np.logspace(-6, 0, 10), "clf__kernel": ["linear"]},
    {"clf__hidden_layer_sizes": [*product([16, 32, 64], [16, 32, 64])], 'clf__learning_rate_init': np.logspace(-4, 0, 4), 'clf__alpha': np.logspace(-4, 0, 4)},
    {"pca__n_components": [5, 10, 25, 50, 100], "clf__hidden_layer_sizes": [*product([16, 32, 64], [16, 32, 64])], 'clf__learning_rate_init': np.logspace(-4, 0, 4), 'clf__alpha': np.logspace(-4, 0, 4)},
    {"clf__n_estimators": [100, 200]},
    {"pca__n_components": [5, 10, 25, 50, 100], "clf__n_estimators": [100, 200], "clf__max_depth": [2, 3, 5, None], "clf_min_samples_leaf": [1, 3, 5, 10, 20]}
]

In [ ]:
with open(f"D:/data/other/{DATA_TYPE}-scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [ ]:
results = []
for (name, pipe), grid in zip(steps, grids):
    if name.endswith('-pca'):
        continue
    print(f"@{name}")
    pipe = [('selector', SelectKBest(mutual_info_classif))] + pipe
    grid['selector__k'] = np.logspace(0.7, np.log10(F), 5).astype(int)
    pipeline = Pipeline(pipe)
    grid = GridSearchCV(pipeline, grid, cv=5, scoring="f1_macro", n_jobs=-1)
    grid.fit(Xtr, ytrain)
    pd.DataFrame(grid.cv_results_).sort_values("rank_test_score").to_csv(f"D:/data/other/{DATA_TYPE}-{name}.csv")
    
    clf = grid.best_estimator_
    clf.fit(Xtr, ytrain)
    with open(f"D:/data/other/{DATA_TYPE}-{name}.pkl", "wb") as f:
        pickle.dump(clf, f)
    y_tr_hat = clf.predict(Xtr)
    y_te_hat = clf.predict(Xte)
    res = {"name": name}
    for phase, pred, true in [("train", y_tr_hat, ytrain), ("test", y_te_hat, ytest)]:
        for f in [accuracy_score, precision_score, recall_score, f1_score]:
            try: 
                res[f"{phase}-{f.__name__}"] = f(true, pred, average="macro")
            except:
                res[f"{phase}-{f.__name__}"] = f(true, pred)
        
    results.append(res)
    print(json.dumps(res, indent=2))
    print("-"*50)
    print()

### XGBOOST With Combined Features

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
ytrain = le.fit_transform(ytrain)
ytest = le.fit_transform(ytest)

In [ ]:
from xgboost import XGBClassifier
name = "xgb"
pipe = [('clf', XGBClassifier(n_estimators=100, objective='multi:softmax', n_jobs=-1,
                    silent=True, nthread=4))]
grid = {
        'clf__n_estimators': [100, 200, 300, 400, 500],
        'clf__max_depth': [3, 4, 5],
        'clf__subsample': [0.5, 0.6, 0.7, 0.8, 0.9],
        'clf__colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
        'clf__min_child_weight': [1, 2, 3, 4, 5],
        'clf__reg_alpha':[0, 0.001, 0.005, 0.01, 0.05],
        'clf__learning_rate': [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
        }

In [ ]:
pipe = [('selector', SelectKBest(mutual_info_classif))] + pipe
grid['selector__k'] = np.logspace(0.7, np.log10(F), 5).astype(int)
pipeline = Pipeline(pipe)
try:
    grid = GridSearchCV(pipeline, grid, scoring="f1_macro", n_jobs=-1, verbose=4)
    grid.fit(Xtr, ytrain)
    pd.DataFrame(grid.cv_results_).sort_values("rank_test_score").to_csv(f"D:/data/other/{DATA_TYPE}-{name}.csv")
    clf = grid.best_estimator_
    clf.fit(Xtr, ytrain)
    with open(f"D:/data/other/{DATA_TYPE}-{name}.pkl", "wb") as f:
        pickle.dump(clf, f)
    y_tr_hat = clf.predict(Xtr)
    y_te_hat = clf.predict(Xte)
    res = {"name": name}
    for phase, pred, true in [("train", y_tr_hat, ytrain), ("test", y_te_hat, ytest)]:
        for f in [accuracy_score, precision_score, recall_score, f1_score]:
            try: 
                res[f"{phase}-{f.__name__}"] = f(true, pred, average="macro")
            except:
                res[f"{phase}-{f.__name__}"] = f(true, pred)
    print(json.dumps(res, indent=2))
    print("-"*50)
    print()
except Exception as e:
    print(str(e))